In [1]:
from transformers import BertTokenizer, BertModel
model_path = '../model/bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)

/root/miniconda3/envs/bigcn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text = ["@Trump @Obama @Biden", "hello \u2345"]
encoded_input = tokenizer(text, return_tensors='pt', padding=True)
output = model(**encoded_input)

In [3]:
print(output.last_hidden_state.shape)

torch.Size([2, 9, 768])


In [4]:
from rumorDataset import RumorStanceDataset, RumorDataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('../model/bert-base-cased')

dataset = RumorStanceDataset('../datasets/semeval2017-task8/', 'traindev', tokenizer)

In [5]:
loader = DataLoader(dataset, collate_fn=RumorStanceDataset.collate_fn)

In [6]:
for node_token, edge_index_TD, edge_index_BU, root_index, rumor_label, stance_label in loader:
    # print(node_token)
    # print(edge_index_TD)
    # print(edge_index_BU)
    # print(root_index)
    print(rumor_label)
    print(stance_label)
    break

tensor([2])
tensor([0, 3, 3, 3, 0])


In [7]:
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from MSABiGCN import BertMSA
import torch
from torch.nn.functional import softmax
bert_model = BertModel.from_pretrained('../model/bert-base-cased/')
device = torch.device('cuda')
model = BertMSA(
    bert_model = bert_model,
    embedding_dim = 768,
    numStanceTag = 4,
    batch_first = True,
    device = device
)

bert_params_id = list(map(id, model.bert_model.parameters()))
bert_params = filter(lambda p:id(p) in bert_params, model.parameters())
base_params = filter(lambda p:id(p) not in bert_params, model.parameters())

from torch import optim
loss_func = torch.nn.CrossEntropyLoss(reduction='mean').to(device)
optimizer = optim.AdamW([
    {'params': base_params},
    {'params': bert_params}
], lr=1e-3)

In [8]:
epoch = 1
model.train()
stanceTrth = []
stancePredict = []
totalLoss = 0.
for node_token, edge_index_TD, edge_index_BU, root_index, rumor_label, stance_label in tqdm(
    iter(loader), 
    desc="[epoch: {:d}] ".format(epoch), 
    leave=False, 
    ncols=100
):
    stance_label = stance_label.to(device)
    stanceTrth += stance_label.tolist()

    optimizer.zero_grad()
    result = model.forward(node_token)
    loss = loss_func(result, stance_label)
    totalLoss += loss
    loss.backward()
    optimizer.step()
    
    result = softmax(result, dim=1)
    stancePredict += result.max(dim=1)[1].tolist()

In [ ]:
stancePredict